In [4]:
import torch
from torch import nn

In [1]:
!nvidia-smi

Mon Oct 23 16:57:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10          Off  | 00000000:00:08.0 Off |                    0 |
|  0%   26C    P8     8W / 150W |      0MiB / 22731MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
torch.cuda.device_count()

1

In [4]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

#### 可以在请求GPU不存在的情况下运行代码

In [6]:
# 尝试使用gpu
def try_gpu(i=0):
    if torch.cuda.device_count() >= i+1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

# 尝试使用所有可用gpu
def try_all_gpus():
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [9]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [7]:
Y = torch.rand(2, 3, device=try_gpu())
Y

tensor([[0.1501, 0.3104, 0.0896],
        [0.7144, 0.3923, 0.8884]], device='cuda:0')

In [8]:
Z = X.cuda(0)
Z

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [9]:
Y + Z

tensor([[1.1501, 1.3104, 1.0896],
        [1.7144, 1.3923, 1.8884]], device='cuda:0')

In [10]:
# 同一gpu下不会重新分配内存
X is Z

True

In [11]:
X = X + Y
X, Z

(tensor([[1.1501, 1.3104, 1.0896],
         [1.7144, 1.3923, 1.8884]], device='cuda:0'),
 tensor([[1., 1., 1.],
         [1., 1., 1.]], device='cuda:0'))

In [7]:
# 神经网络
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [12]:
net(X)

tensor([[0.0534],
        [0.0534]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [14]:
net[0].weight.data.device

device(type='cuda', index=0)